<a href="https://colab.research.google.com/github/melisnv/ma-course-subjectivity-mining/blob/master/subjectivity_mining_assignmentIII.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
#!pip install simpletransformers

In [2]:
# importing the task-specific model
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix,average_precision_score



In [3]:
data_olid_train = pd.read_csv("/content/datas/olid-train.csv")
data_olid_train.head()

,id,text,labels
0,86426,@USER She should ask a few native Americans wh...,1
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,1
2,16820,Amazon is investigating Chinese employees who ...,0
3,62688,"@USER Someone should'veTaken"" this piece of sh...",1
4,43605,@USER @USER Obama wanted liberals &amp; illega...,0


In [4]:
data_olid_test = pd.read_csv("/content/datas/olid-test.csv")
data_olid_test.head()

,id,text,labels
0,15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,1
1,27014,"#ConstitutionDay is revered by Conservatives, ...",0
2,30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,0
3,13876,#Watching #Boomer getting the news that she is...,0
4,60133,#NoPasaran: Unity demo to oppose the far-right...,1


In [5]:
data_hasoc = pd.read_csv("/content/datas/hasoc-train.csv")
data_hasoc.head()

,id,text,labels
0,hasoc_en_1,#DhoniKeepsTheGlove | WATCH: Sports Minister K...,0
1,hasoc_en_2,@politico No. We should remember very clearly ...,1
2,hasoc_en_3,@cricketworldcup Guess who would be the winner...,0
3,hasoc_en_4,Corbyn is too politically intellectual for #Bo...,0
4,hasoc_en_5,All the best to #TeamIndia for another swimmin...,0


In [6]:
# loading a pre-trained model

model_args = ClassificationArgs()
model_args.num_train_epochs = 1
# model_args.learning_rate = 1e-4
#model_args.num_train_epochs = 5 # default
model_args.overwrite_output_dir= True
model_args.output_dir = '/content/drive/MyDrive/NLP/trained_models_bert'

model_bert = ClassificationModel("bert", "bert-base-cased", args=model_args)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [7]:
model_bert

In [8]:
data_olid_train.isnull().sum()

id        0
text      0
labels    0
dtype: int64

# In-domain Experiments

In [9]:
model_bert.train_model(data_olid_train)

  0%|          | 0/13240 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/1655 [00:00<?, ?it/s]

(1655, 0.48784018715342725)

In [10]:
result, model_outputs, wrong_predictions = model_bert.eval_model(data_olid_test, report = classification_report)

  0%|          | 0/860 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/108 [00:00<?, ?it/s]

In [11]:
print(wrong_predictions)

[{'guid': 0, 'text_a': '#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA Democrats Support Antifa, Muslim Brotherhood, MS13, ISIS, Pedophilia, Child Trafficking, Taxpayer Funded Abortion’s, Election Fraud, Sedition And Treason!!! #LockThemAllUp #WWG1WGA #QAnon \u2066@USER URL', 'text_b': None, 'label': 1}, {'guid': 4, 'text_a': '#NoPasaran: Unity demo to oppose the far-right in #London – #antifa #Oct13 — Enough is Enough! URL', 'text_b': None, 'label': 1}, {'guid': 6, 'text_a': '#RAP is a form of ART! Used to express yourself freely. It does not gv the green light or excuse the behavior of acting like an animal! She is not in the streets of the BX where violence is a way of living. Elevate yourself boo and get on @USER level for longevity! #QUEEN👑', 'text_b': None, 'label': 0}, {'guid': 7, 'text_a': '@USER Do you get the feeling he is kissing @USER behind so he can humiliate him later?', 'text_b': None, 'label': 1}, {'guid': 21, 'text_a': '#Kavanaugh so a wild claim from 36 years ago of 

In [12]:
print(result['report'])

              precision    recall  f1-score   support

         0.0       0.87      0.93      0.90       620
         1.0       0.77      0.65      0.71       240

    accuracy                           0.85       860
   macro avg       0.82      0.79      0.80       860
weighted avg       0.85      0.85      0.85       860



In [13]:
predictions, raw_outputs = model_bert.predict(list(data_olid_test['text']))

  0%|          | 0/860 [00:00<?, ?it/s]

  0%|          | 0/108 [00:00<?, ?it/s]

In [14]:
predicted_labels = pd.DataFrame({'text':data_olid_test.text, 'predicted': predictions})
predicted_labels.head()

,text,predicted
0,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,0
1,"#ConstitutionDay is revered by Conservatives, ...",0
2,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,0
3,#Watching #Boomer getting the news that she is...,0
4,#NoPasaran: Unity demo to oppose the far-right...,0


In [15]:
predicted_labels.to_csv('predicted_labels_bert.csv', index=False)

In [16]:
print("Bert Confusion Matrix for OLID Train Dataset:: ")
print(confusion_matrix(data_olid_test['labels'], predictions))

Bert Confusion Matrix for OLID Train Dataset:: 
[[574  46]
 [ 83 157]]


## Roberta Model

In [17]:
model_roberta_args = ClassificationArgs()
# model_args.num_train_epochs = 5 # default
# model_args.learning_rate = 1e-4
model_roberta_args.num_train_epochs = 1
model_roberta_args.overwrite_output_dir= True
model_roberta_args.output_dir = '/content/drive/MyDrive/NLP/trained_models_roberta'

In [18]:
model_roberta = ClassificationModel("roberta", "roberta-base", args=model_roberta_args)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [19]:
model_roberta.train_model(data_olid_train)

  0%|          | 0/13240 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/1655 [00:00<?, ?it/s]

(1655, 0.5047604543564903)

In [20]:
model_roberta

In [21]:
result_2, model_outputs_2, wrong_predictions_2 = model_roberta.eval_model(data_olid_test, report = classification_report)

  0%|          | 0/860 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/108 [00:00<?, ?it/s]

In [22]:
print(wrong_predictions_2)

[{'guid': 0, 'text_a': '#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA Democrats Support Antifa, Muslim Brotherhood, MS13, ISIS, Pedophilia, Child Trafficking, Taxpayer Funded Abortion’s, Election Fraud, Sedition And Treason!!! #LockThemAllUp #WWG1WGA #QAnon \u2066@USER URL', 'text_b': None, 'label': 1}, {'guid': 4, 'text_a': '#NoPasaran: Unity demo to oppose the far-right in #London – #antifa #Oct13 — Enough is Enough! URL', 'text_b': None, 'label': 1}, {'guid': 14, 'text_a': '@USER Nigga ware da hits at', 'text_b': None, 'label': 1}, {'guid': 25, 'text_a': '#HIAC  Damn Matt Hardy and Randy Orton put on one hell in a cell match!! Woooo!!! I hope he is okay!!', 'text_b': None, 'label': 0}, {'guid': 30, 'text_a': '#ChristineBlaseyFord is your #Kavanaugh accuser... #Liberals try this EVERY time... #ConfirmJudgeKavanaugh URL', 'text_b': None, 'label': 1}, {'guid': 41, 'text_a': '#Georgetown Classmate Says #Left Accuser is Absolutely NUTS"  URL "Judge Kavanaugh"@USER @USER @USER #ABC #CBS 

In [23]:
predictions_rob, raw_outputs_rob = model_roberta.predict(list(data_olid_test['text']))
predicted_labels_rob = pd.DataFrame({'text':data_olid_test.text, 'predicted': predictions_rob})
predicted_labels_rob.head()

  0%|          | 0/860 [00:00<?, ?it/s]

  0%|          | 0/108 [00:00<?, ?it/s]

,text,predicted
0,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,0
1,"#ConstitutionDay is revered by Conservatives, ...",0
2,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,0
3,#Watching #Boomer getting the news that she is...,0
4,#NoPasaran: Unity demo to oppose the far-right...,0


In [24]:
predicted_labels_rob.to_csv('predicted_labels_roberta.csv', index=False)


In [25]:
print("Roberta Confusion Matrix for OLID Train Dataset: ")
print(confusion_matrix(data_olid_test['labels'], predictions_rob))

Roberta Confusion Matrix for OLID Train Dataset: 
[[574  46]
 [ 96 144]]


# Cross-domain experiments

In [26]:
model_bert.train_model(data_hasoc)
result_3, model_outputs_3, wrong_predictions_3 = model_bert.eval_model(data_olid_test, report = classification_report)

print(result['report'])

  0%|          | 0/5852 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/732 [00:00<?, ?it/s]

  0%|          | 0/860 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/108 [00:00<?, ?it/s]

              precision    recall  f1-score   support

         0.0       0.87      0.93      0.90       620
         1.0       0.77      0.65      0.71       240

    accuracy                           0.85       860
   macro avg       0.82      0.79      0.80       860
weighted avg       0.85      0.85      0.85       860



In [27]:
predictions_3, raw_outputs_3 = model_bert.predict(list(data_olid_test['text']))
predicted_labels_bert_hasoc = pd.DataFrame({'text':data_olid_test.text, 'predicted': predictions_3})
predicted_labels_bert_hasoc.head()

  0%|          | 0/860 [00:00<?, ?it/s]

  0%|          | 0/108 [00:00<?, ?it/s]

,text,predicted
0,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,0
1,"#ConstitutionDay is revered by Conservatives, ...",0
2,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,0
3,#Watching #Boomer getting the news that she is...,0
4,#NoPasaran: Unity demo to oppose the far-right...,0


In [28]:
predicted_labels_bert_hasoc.to_csv('predicted_labels_bert_hasoc.csv', index=False)

In [29]:
print("Bert Confusion Matrix for Hosac Train Dataset: ")
print(confusion_matrix(data_olid_test['labels'], predictions_rob))

Bert Confusion Matrix for Hosac Train Dataset: 
[[574  46]
 [ 96 144]]


## Roberta on Hasoc Train Data

In [30]:
model_roberta.train_model(data_hasoc)
result_rob_has, model_outputs_rob_has, wrong_predictions_rob_has = model_roberta.eval_model(data_olid_test, 
                                                                                            report = classification_report)

print(result['report'])

  0%|          | 0/5852 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/732 [00:00<?, ?it/s]

  0%|          | 0/860 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/108 [00:00<?, ?it/s]

              precision    recall  f1-score   support

         0.0       0.87      0.93      0.90       620
         1.0       0.77      0.65      0.71       240

    accuracy                           0.85       860
   macro avg       0.82      0.79      0.80       860
weighted avg       0.85      0.85      0.85       860



In [32]:
predictions_rob_has, raw_outputs_rob_has = model_roberta.predict(list(data_olid_test['text']))
predicted_labels_roberta_hasoc = pd.DataFrame({'text':data_olid_test.text, 'predicted': predictions_rob_has})
predicted_labels_roberta_hasoc.head()

  0%|          | 0/860 [00:00<?, ?it/s]

  0%|          | 0/108 [00:00<?, ?it/s]

,text,predicted
0,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,1
1,"#ConstitutionDay is revered by Conservatives, ...",0
2,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,0
3,#Watching #Boomer getting the news that she is...,0
4,#NoPasaran: Unity demo to oppose the far-right...,0


In [33]:
predicted_labels_roberta_hasoc.to_csv('predicted_labels_roberta_hasoc.csv', index=False)

In [34]:
print("Bert Confusion Matrix for Hosac Train Dataset: ")
print(confusion_matrix(data_olid_test['labels'], predictions_rob_has))

Bert Confusion Matrix for Hosac Train Dataset: 
[[571  49]
 [ 98 142]]
